In [ ]:
import nibabel as nib
import numpy as np
import ipdb
import os
import csv
import torch
from nilearn import plotting
from skimage.transform import resize
from ipdb import set_trace

In [ ]:
arr = range(0, 256, 14)
path = '../ADNI/002_S_0295/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-04-18_08_20_30.0/S13408/mri'
# path = '../ADNI/002_S_0619/FreeSurfer_Cross-Sectional_Processing_brainmask/2007-06-22_07_06_17.0/S33969/mri'
mgz_path = rf'{path}/brainmask.mgz'
nii_path = rf'{path}/test_image.nii'
padding = 0

img = nib.load(mgz_path)
np_img = np.array(img.get_fdata())
plotting.plot_img(img,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
plotting.plot_img(img,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
plotting.plot_img(img,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
plotting.show()
arr = range(0, img.shape[0], img.shape[0] // 15)

def clean_axis(np_img, path, axis=0):
    x_len, y_len, z_len = len(np_img), len(np_img[0]), len(np_img[0, 0])
    zero_mask = np.zeros(0)
    del_inds = []
    save_i = -1
    iter_len = 0
    continuity_warning = 0
    
    if axis == 0: 
        zero_mask = np.zeros((y_len, z_len))
        iter_len = x_len
    elif axis == 1: 
        zero_mask = np.zeros((x_len, z_len))
        iter_len = y_len
    elif axis == 2: 
        zero_mask = np.zeros((x_len, y_len))
        iter_len = z_len
    for i in range(iter_len):
        plane = np.sum(np_img[i] != zero_mask) if axis == 0 else np.sum(np_img[:, i] != zero_mask) if axis == 1 else np.sum(np_img[:, :, i] != zero_mask) 
        if plane == 0: 
            del_inds.append(i)
            if save_i + 1 != i: continuity_warning += 1
            save_i = i

    # cannot allow two breakpoints in del_inds
    if continuity_warning >= 2: print(f'\x1b[31m broken del_inds: {path} \n axis: {axis} \x1b[0m') 
        
    # remove empty slices
    np_img = np.delete(np.swapaxes(np_img, 0, axis), del_inds, 0)
    return np.swapaxes(np_img, 0, axis)

    # add padding
    #     np_img = np.insert(np_img, [0 for i in range(padding)], zero_mask, 0)
    #     np_img = np.insert(np_img, [len(np_img) for i in range(padding)], zero_mask, 0)
    
# clean the 0-intensity space surrounding the brain
def clean_space(root, f):
    mgz_path = rf'{root}/{f}'
    nii_path = rf'{root}/image.nii'
    img = nib.load(mgz_path)
    np_img = np.array(img.get_fdata())
    arr = range(0, 64, 64 // 15)
    
    #clean axes
#     np_img = clean_axis(np_img, mgz_path, axis=0)
#     np_img = clean_axis(np_img, mgz_path, axis=1)
#     np_img = clean_axis(np_img, mgz_path, axis=2)
    
    # write to disk
    feat = nib.Nifti1Image(np_img, img.affine, img.header)
    nib.save(feat, nii_path)

    #show image
    img = nib.load(nii_path)
    np_img = np.array(img.get_fdata())
    data = resize(np_img, (64, 64, 64), mode='constant')
    data_nii = nib.Nifti1Image(data, np.eye(4))
    plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
    plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
    plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
    plotting.show()

    
clean_space(path, 'brainmask.mgz')